In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1598209757476_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-1>

In [2]:
#Specifying the schema instead of inferring it 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, DoubleType, LongType

fileSchema = StructType([StructField('movie_id', IntegerType(),True),
                        StructField('user_id', IntegerType(),True),
                        StructField('rating', IntegerType(),True),
                        StructField('time_stamp', LongType(),True)])

movie_ratings = spark.read.load("s3://sparkbucket123/u.data", format="csv", sep="\t", schema = fileSchema, header="true")

movie_ratings.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+------+----------+
|movie_id|user_id|rating|time_stamp|
+--------+-------+------+----------+
|     186|    302|     3| 891717742|
|      22|    377|     1| 878887116|
|     244|     51|     2| 880606923|
|     166|    346|     1| 886397596|
|     298|    474|     4| 884182806|
|     115|    265|     2| 881171488|
|     253|    465|     5| 891628467|
|     305|    451|     3| 886324817|
|       6|     86|     3| 883603013|
|      62|    257|     2| 879372434|
|     286|   1014|     5| 879781125|
|     200|    222|     5| 876042340|
|     210|     40|     3| 891035994|
|     224|     29|     3| 888104457|
|     303|    785|     3| 879485318|
|     122|    387|     5| 879270459|
|     194|    274|     2| 879539794|
|     291|   1042|     4| 874834944|
|     234|   1184|     2| 892079237|
|     119|    392|     4| 886176814|
+--------+-------+------+----------+
only showing top 20 rows

In [3]:

movies = spark.read.load("s3://sparkbucket123/u.item", format="csv", sep="|", inferSchema="true", header="true")

movies.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+-----------+----+-----------------------------------------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| 10|    Toy Story (1995)|01-Jan-1995| _c3|http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)| 05| 06| 07| 18| 19|110|011|012|013|014|015|016|017|018|019|020|021|022|023|
+---+--------------------+-----------+----+-----------------------------------------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|  2|    GoldenEye (1995)|01-Jan-1995|null|                                 http://us.imdb.co...|  0|  1|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  1|  0|  0|
|  3|   Four Rooms (1995)|01-Jan-1995|null|                                 http://us.imdb.co...|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  1|  0|  0|
|  4|   Get Shorty (1995)|01-Jan-1995|null|                                 http://us.imdb.co...|  0|  1|  0|  0|  0|  1|  0|  0| 

In [ ]:
movies.show(1)

In [4]:
movie_selected = movies.select('10', 'Toy Story (1995)')
from functools import reduce

oldColumns = movie_selected.schema.names
newColumns = ["movie_id", "movie_name"]

movie_names = reduce(lambda movie_selected, idx: movie_selected.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), movie_selected)
movie_names.printSchema()
movie_names.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- movie_id: integer (nullable = true)
 |-- movie_name: string (nullable = true)

+--------+--------------------+
|movie_id|          movie_name|
+--------+--------------------+
|       2|    GoldenEye (1995)|
|       3|   Four Rooms (1995)|
|       4|   Get Shorty (1995)|
|       5|      Copycat (1995)|
|       6|Shanghai Triad (Y...|
|       7|Twelve Monkeys (1...|
|       8|         Babe (1995)|
|       9|Dead Man Walking ...|
|      10|  Richard III (1995)|
|      11|Seven (Se7en) (1995)|
|      12|Usual Suspects, T...|
|      13|Mighty Aphrodite ...|
|      14|  Postino, Il (1994)|
|      15|Mr. Holland's Opu...|
|      16|French Twist (Gaz...|
|      17|From Dusk Till Da...|
|      18|White Balloon, Th...|
|      19|Antonia's Line (1...|
|      20|Angels and Insect...|
|      21|Muppet Treasure I...|
+--------+--------------------+
only showing top 20 rows

In [5]:
shuffled_join = movie_ratings.join(movie_names , ["movie_id"])
shuffled_join.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+------+----------+--------------------+
|movie_id|user_id|rating|time_stamp|          movie_name|
+--------+-------+------+----------+--------------------+
|     186|    302|     3| 891717742|Blues Brothers, T...|
|      22|    377|     1| 878887116|   Braveheart (1995)|
|     244|     51|     2| 880606923|Smilla's Sense of...|
|     166|    346|     1| 886397596|Manon of the Spri...|
|     298|    474|     4| 884182806|     Face/Off (1997)|
|     115|    265|     2| 881171488|Haunted World of ...|
|     253|    465|     5| 891628467|Pillow Book, The ...|
|     305|    451|     3| 886324817|Ice Storm, The (1...|
|       6|     86|     3| 883603013|Shanghai Triad (Y...|
|      62|    257|     2| 879372434|     Stargate (1994)|
|     286|   1014|     5| 879781125|English Patient, ...|
|     200|    222|     5| 876042340| Shining, The (1980)|
|     210|     40|     3| 891035994|Indiana Jones and...|
|     224|     29|     3| 888104457|     Ridicule (1996)|
|     303|    

In [6]:
from pyspark.sql.functions import broadcast

joined_data = movie_ratings.join(movie_names.hint("broadcast") , ["movie_id"])

joined_data.show()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+------+----------+--------------------+
|movie_id|user_id|rating|time_stamp|          movie_name|
+--------+-------+------+----------+--------------------+
|     186|    302|     3| 891717742|Blues Brothers, T...|
|      22|    377|     1| 878887116|   Braveheart (1995)|
|     244|     51|     2| 880606923|Smilla's Sense of...|
|     166|    346|     1| 886397596|Manon of the Spri...|
|     298|    474|     4| 884182806|     Face/Off (1997)|
|     115|    265|     2| 881171488|Haunted World of ...|
|     253|    465|     5| 891628467|Pillow Book, The ...|
|     305|    451|     3| 886324817|Ice Storm, The (1...|
|       6|     86|     3| 883603013|Shanghai Triad (Y...|
|      62|    257|     2| 879372434|     Stargate (1994)|
|     286|   1014|     5| 879781125|English Patient, ...|
|     200|    222|     5| 876042340| Shining, The (1980)|
|     210|     40|     3| 891035994|Indiana Jones and...|
|     224|     29|     3| 888104457|     Ridicule (1996)|
|     303|    